In [27]:
import os
import pandas as pd
import requests
import datetime as dt

In [28]:
noaa_source_spreadsheets = 'https://www.glerl.noaa.gov/data/dashboard/data/hydroIO/sourceSpreadsheets/'

In [29]:
#ice data
path = os.path.join('..','data_files','ice_data')
erie = 'erieIceData.csv'
huron = 'huronIceData.csv'
mi = 'miIceData.csv'
ontario = 'ontarioIceData.csv'
superior = 'superiorIceData.csv'
files = [erie,huron,mi,ontario,superior]
df = pd.DataFrame()
sources = []

In [30]:
for e in files:
    new_df = pd.read_csv(os.path.join(path,e))
    lake = new_df.columns[0].replace(':','')
    sources.append({'Lake':lake,
                   'Sources':new_df.iloc[0]})
    new_df.columns = ['Date',f'{lake}']
    new_df = new_df.drop([0,1])
    if len(df) == 0:
        df = new_df
    else:
        df = df.merge(new_df,how='outer',left_on='Date',right_on ='Date')
df.head()

,Date,Lake Erie,Lake Huron,Lake Michigan,Lake Ontario,Lake Superior
0,12/20/1972,0,25.26,9.21,0.61,16.62
1,12/28/1972,0,12.65,12.09,2.98,9.35
2,01/02/1973,3.4,22.0,6.53,2.8,13.03
3,01/09/1973,25.4,31.3,21.55,21.4,29.69
4,01/16/1973,42.3,25.3,22.39,11.7,15.25


Covert Date column to DateTime
Should we make that column the index?

In [31]:
df['Date'] = pd.to_datetime(df['Date'])
type(df['Date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [32]:
df

,Date,Lake Erie,Lake Huron,Lake Michigan,Lake Ontario,Lake Superior
0,1972-12-20,0,25.26,9.21,0.61,16.62
1,1972-12-28,0,12.65,12.09,2.98,9.35
2,1973-01-02,3.4,22.0,6.53,2.8,13.03
3,1973-01-09,25.4,31.3,21.55,21.4,29.69
4,1973-01-16,42.3,25.3,22.39,11.7,15.25
...,...,...,...,...,...,...
2519,2013-11-23,NaN,NaN,NaN,NaN,0
2520,2014-06-06,NaN,NaN,NaN,NaN,0
2521,2014-11-13,NaN,NaN,NaN,NaN,0
2522,2015-05-29,NaN,NaN,NaN,NaN,0


In [33]:
df[df['Date']>'2000-01-01']

,Date,Lake Erie,Lake Huron,Lake Michigan,Lake Ontario,Lake Superior
956,2000-01-07,5.7,8.9,3.84,1.7,3.26
957,2000-01-14,7.2,15.4,8.78,2.3,6.61
958,2000-01-21,26.7,34.8,16.64,16.4,7.81
959,2000-01-28,80.2,40.4,27.71,17.1,18.4
960,2000-02-04,78.9,29.4,17.61,10.6,11.25
...,...,...,...,...,...,...
2519,2013-11-23,NaN,NaN,NaN,NaN,0
2520,2014-06-06,NaN,NaN,NaN,NaN,0
2521,2014-11-13,NaN,NaN,NaN,NaN,0
2522,2015-05-29,NaN,NaN,NaN,NaN,0


In [34]:
max(df['Date'])

Timestamp('2016-05-05 00:00:00')

In [35]:
min(df['Date'])

Timestamp('1972-12-19 00:00:00')

In [38]:
df = df.sort_values(by='Date')
df[pd.isnull(df).any(axis=1)]

,Date,Lake Erie,Lake Huron,Lake Michigan,Lake Ontario,Lake Superior
2468,1972-12-19,NaN,NaN,NaN,NaN,0
2453,1973-04-07,NaN,0.0,NaN,NaN,0
2454,1974-05-03,NaN,0.0,0,NaN,0
2455,1975-04-25,NaN,0.0,0,NaN,0
72,1976-04-21,0.0,0.0,0,NaN,0
...,...,...,...,...,...,...
2519,2013-11-23,NaN,NaN,NaN,NaN,0
2520,2014-06-06,NaN,NaN,NaN,NaN,0
2521,2014-11-13,NaN,NaN,NaN,NaN,0
2522,2015-05-29,NaN,NaN,NaN,NaN,0


In [45]:
date_diff = []
for i in range(len(df['Date'])-1):
    date_diff.append((df['Date'][i+1]-df['Date'][i]).days)

In [53]:
df['Weekday'] = df['Date'].dt.day_name()

In [54]:
df

,Date,Lake Erie,Lake Huron,Lake Michigan,Lake Ontario,Lake Superior,Weekday
2468,1972-12-19,NaN,NaN,NaN,NaN,0,Tuesday
0,1972-12-20,0,25.26,9.21,0.61,16.62,Wednesday
1,1972-12-28,0,12.65,12.09,2.98,9.35,Thursday
2,1973-01-02,3.4,22.0,6.53,2.8,13.03,Tuesday
3,1973-01-09,25.4,31.3,21.55,21.4,29.69,Tuesday
...,...,...,...,...,...,...,...
2448,2016-05-01,0,0,0,0,0.55,Sunday
2449,2016-05-02,0,0,0,0,0.24,Monday
2450,2016-05-03,0,0,0,0,0.24,Tuesday
2451,2016-05-04,0,0,0,0,0.24,Wednesday


In [55]:
df['Weekday'].value_counts()

Monday       564
Thursday     441
Wednesday    422
Friday       419
Tuesday      291
Saturday     205
Sunday       182
Name: Weekday, dtype: int64